In [1]:
import paramiko
import getpass

host = "158.251.88.197"
port = 22
username = "iaraya"
password = getpass.getpass()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

In [2]:
import random

def write_instance(L,W,H,boxes,filename) :
    txt = "1\n1 0\n"+str(L)+" "+str(W)+" "+str(H)+"\n";
    txt += str(len(boxes)) + "\n"
    for box in boxes:
        txt += str(box.id) + ' ' + str(box.l) + ' ' + str(box.rotx) \
            + ' ' + str(box.w) + ' ' + str(box.roty) \
            + ' ' + str(box.h) + ' ' + str(box.rotz) + ' ' + str(boxes[box])  + '\n'
    txt += '\n'

    text_file = open(filename, "w")
    text_file.write(txt)
    text_file.close()


In [3]:
import os
import json
from scp import SCPClient

def bsg_solve(L,W,H,boxes,id2box, time=1):
    filename="tmp_instance_"+ str(random.randint(10000, 99999))
    
    write_instance(L,W,H,boxes,filename)
    
    scp = SCPClient(ssh.get_transport())
    scp.put(filename, "/home/iaraya/clp/"+filename)

    stdin, stdout, stderr = ssh.exec_command("/home/iaraya/clp/BSG_CLP /home/iaraya/clp/"+filename+" -i 0 -t "+str(time)+" --json")
    lines = stdout.readlines()
    print(lines[-1])
    json_data = json.loads(lines[-1])
    loaded={}; remaining={}
    for item in json_data["loaded"]:
        loaded[id2box[item[0]]]=item[1]
    for item in json_data["remaining"]:
        remaining[id2box[item[0]]]=item[1]
        
    ssh.exec_command("rm /home/iaraya/clp/"+filename)
    os.remove(filename)
    return remaining, loaded, json_data["utilization"]


In [4]:
from box import box

# id l   w g  rotx roty rotz nn
# 1	 108 0 76 0	   30	1	 7

def load_BRinstance(filename, inst=1, nbox = 1):
    boxes={}; id2box={}
    with open(filename) as f:
        n = int(next(f))
        for i in range(n):
            next(f)
            L, W, H =  [int(x) for x in next(f).split()]
            n_boxes = int(next(f))
            for it in range(n_boxes):
                id, l, rotx, w, roty, h, rotz, nn = [int(x) for x in next(f).split()] 
                if i == inst-1:
                    b = box(id, l, w, h, rotx, roty, rotz)
                    # Multiplicar nn para complejizar el problema
                    boxes[b]=nn*nbox;
                    id2box[id]=b
            if i == inst-1: 
                return L,W,H,boxes,id2box

### Ejemplo usando instancia BR

In [9]:
from bin import bin
import numpy as np

solution_size = 0
solution = np.array([])
L,W,H,boxes,id2box = load_BRinstance("BR8.txt",4,10)

L = int(L/2)
W = int(W/2)
H = int(H/2)

#Generating candidate Solution
while(len(boxes) > 0):
    solution_size +=1
    remaining, loaded, utilization = bsg_solve(L,W,H,boxes,id2box,time=5)
    container = bin(solution_size, loaded, utilization)
    solution = np.append(solution, container)
    boxes = remaining

print("Initial Solution: {}".format(len(solution)))

{"remaining" :[[1,16],[2,50],[3,30],[4,76],[5,20],[6,40],[7,70],[8,60],[9,16],[10,60],[11,90],[12,4],[13,30],[14,8],[15,26],[16,39],[17,80],[18,60],[19,40],[20,60],[21,50],[22,48],[23,50],[24,30],[25,26],[26,20],[27,40],[28,60],[29,90],[30,80]], "loaded" :[[1,24],[2,10],[4,4],[9,4],[12,16],[14,22],[15,4],[16,1],[22,2],[25,4]], "utilization" : 0.96346175}

{"remaining" :[[2,28],[3,29],[4,71],[5,20],[6,40],[7,69],[8,60],[9,8],[10,60],[11,90],[13,22],[15,21],[16,39],[17,80],[18,60],[19,26],[20,59],[21,48],[22,48],[23,50],[24,27],[25,24],[26,20],[27,40],[28,52],[29,90],[30,80]], "loaded" :[[1,16],[2,22],[3,1],[4,5],[7,1],[9,8],[12,4],[13,8],[14,8],[15,5],[19,14],[20,1],[21,2],[24,3],[25,2],[28,8]], "utilization" : 0.96315148}

{"remaining" :[[2,17],[3,18],[4,68],[5,20],[6,40],[7,67],[8,59],[9,8],[10,60],[11,90],[13,7],[15,21],[16,33],[17,78],[18,58],[19,18],[20,59],[21,42],[22,42],[23,42],[24,24],[25,23],[26,20],[27,37],[28,36],[29,82],[30,76]], "loaded" :[[2,11],[3,11],[4,3],[7,2],[8,1],[

In [14]:
solution_size = 0
solution = np.array([])
L,W,H,boxes,id2box = load_BRinstance("BR8.txt",4, 10)
L = int(L/2)
W = int(W/2)
H = int(H/2) 
Vmax = L*W*H

#Generating candidate Solution
while(len(boxes) > 0):
    solution_size +=1
    
    vol_c = 0
    c = dict()
    while(vol_c < 1.5*Vmax and len(boxes) != 0):
        boxes_keys = list(boxes.keys())
        r =  random.randint(0, len(boxes_keys)-1)
        #print(r, len(boxes_keys)-1)
        b = boxes_keys[r]
        n = min(boxes[b],8)    
        boxes[b] = boxes[b]-n

        if(boxes[b] == 0):
            boxes.pop(b)

        if(b not in c):
            c[b] = n
        else:
            c[b] = c[b]+n
        
        vol_box = b.vol
        vol_c = vol_c + vol_box*n

    remaining, loaded, utilization = bsg_solve(L,W,H, c ,id2box,time=5)
    container = bin(solution_size, loaded, utilization)
    solution = np.append(solution, container)
    print(remaining, boxes)
    for key in remaining:
        if(key in boxes):
            boxes[key] += remaining[key]
        else:
            boxes[key] = remaining[key]

print("Initial Solution: {}".format(len(solution)))

 at 0x7fbbe4c23dd0>: 8, <box.box object at 0x7fbbe4c23c50>: 14, <box.box object at 0x7fbbe4c23c10>: 13, <box.box object at 0x7fbbe4c23c90>: 1}
{"remaining" :[[21,6],[28,23],[14,23],[19,8],[7,1],[12,16],[9,1],[29,6]], "loaded" :[[21,2],[28,1],[14,1],[27,2],[29,2],[4,8]], "utilization" : 0.87275937}

{<box.box object at 0x7fbbe4c23cd0>: 6, <box.box object at 0x7fbbe4c23e90>: 23, <box.box object at 0x7fbbe4c23b10>: 23, <box.box object at 0x7fbbe4c23c50>: 8, <box.box object at 0x7fbbe4c23910>: 1, <box.box object at 0x7fbbe4c23a90>: 16, <box.box object at 0x7fbbe4c23990>: 1, <box.box object at 0x7fbbe4c23ed0>: 6} {<box.box object at 0x7fbbe4e9b610>: 6, <box.box object at 0x7fbbe4c23950>: 20, <box.box object at 0x7fbbe4c23bd0>: 76, <box.box object at 0x7fbbe4c23cd0>: 13, <box.box object at 0x7fbbe4c23e90>: 35, <box.box object at 0x7fbbe4c23ed0>: 74, <box.box object at 0x7fbbe4c23f10>: 3, <box.box object at 0x7fbbe4c23b10>: 3, <box.box object at 0x7fbbe4c238d0>: 17, <box.box object at 0x7fbbe

### Ejemplo creando instancia y resolviendo

In [7]:
boxes={}
b=box(1,10,10,10,1,1,1)
boxes[b]=5; id2box[1]=b # 5 cajas de 10x10x10 
b=box(2,20,20,20,1,1,1)
boxes[b]=5; id2box[2]=b # 5 cajas de 20x20x20 
b=box(3,40,40,40,1,1,1)
boxes[b]=5; id2box[3]=b # 5 cajas de 40x40x40 
b=box(4,80,80,80,1,1,1)
boxes[b]=5; id2box[4]=b # 5 cajas de 80x80x80 

remaining, loaded, utilization = bsg_solve(100,100,100,boxes,id2box, time=1)

{"remaining" :[[3,5],[4,4]], "loaded" :[[1,5],[2,5],[4,1]], "utilization" : 0.557}

